In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import KernelPCA
import plotly.express as px
from flair.embeddings import TransformerWordEmbeddings

from evolvemb import EvolvingEmbeddings, DummyEmbeddings
from emb_noflair import SimplePretrainedEmbeddings

%load_ext autoreload
%autoreload 2

In [ ]:
def load_nyt(start_date="2019-01-01", end_date="2020-12-31"):
    # read in NYT dataset
    sentences = []
    dates = []
    with open("data/nytimes_dataset.txt") as f:
        for line in f:
            d, s = line.strip().split("\t")
            if d < start_date:
                continue
            elif d > end_date:
                break
            dates.append(d)
            # lowercase! and some longer words mistakenly can end with "." due to the tokenizer; remove this!
            sentences.append([w if len(w) <= 3 or not w.endswith(".") else w[:-1] for w in s.lower().split()])
    print("Dataset contains %i sentences between %s and %s" % (len(sentences), start_date, end_date))
    return sentences, dates


def get_emb_snapshots(snapshots, start_date="2019-01-01", min_freq=100, n_tokens=10000, saveemb=False):
    savepath = "snapshot_emb_%s_%s_%i.pkl" % (start_date, snapshots[-1], min_freq)
    # see if we can just load the embeddings
    if os.path.exists(savepath):
        try:
            snapshot_emb = pickle.load(open(savepath, "rb"))
            return snapshot_emb
        except:
            pass
    # load dataset
    sentences, dates = load_nyt(start_date, snapshots[-1])
    # transformer model to generate the local embeddings
    # local_emb = DummyEmbeddings(50, "testemb")  # for testing of plots only
    local_emb = TransformerWordEmbeddings("bert-base-uncased", layers="all", use_scalar_mix=True, pooling_operation="mean", fine_tune=False)
    # pass sentences directly to generate input model from all texts 
    # so we know which words are of interest and their count (to set alphas manually afterwards)
    emb = EvolvingEmbeddings(local_emb, sentences, alpha=None, min_freq=min_freq, n_tokens=n_tokens, update_index=False)
    print("Number of words in the vocabulary (for which we'll learn an embedding):", emb.input_model.n_tokens)
    # create counts dict based on token_counts/split
    counts_dict = {t: emb.input_model.token_counts[t]/(1.*len(snapshots)) for t in emb.input_model.index2token}
    # manually create max_counts array with entries for individual words
    emb._set_max_count(counts_dict)
    # compute evolving embeddings and take snapshot at the end of each split
    snapshot_emb = {}
    current_snap = 0
    for i, s in enumerate(sentences):
        if not i % 100:
            print("Processing sentence %8i/%i" % (i, len(sentences)), end="\r")
        # check if we need to take the snapshot
        if dates[i] > snapshots[current_snap]:
            actual_snap = dates[i-1]  # the given snapshot might be at the 31st but the month could have only 30 days
            temp = emb.as_pretrained()
            # save as a model without flair dependency
            snapshot_emb[actual_snap] = SimplePretrainedEmbeddings(temp.embeddings, temp.input_model)
            # set OOV embedding to zeros
            snapshot_emb[actual_snap].embeddings[-1] = np.zeros(snapshot_emb[actual_snap].embeddings.shape[1])
            current_snap += 1
        # update embeddings with sentence
        emb.update_evolving_embeddings(s)
    # possibly take last snapshot
    if current_snap < len(snapshots):
        actual_snap = dates[i]
        temp = emb.as_pretrained()
        snapshot_emb[actual_snap] = SimplePretrainedEmbeddings(temp.embeddings, temp.input_model)
        snapshot_emb[actual_snap].embeddings[-1] = np.zeros(snapshot_emb[actual_snap].embeddings.shape[1])
    print("Processing sentence %8i/%i...done!" % (len(sentences), len(sentences)))
    # reduce file size by ensuring dtype of numpy arrays is float32; transform into objects with less dependencies
    for s in snapshot_emb:
        snapshot_emb[s].embeddings = np.array(snapshot_emb[s].embeddings, dtype=np.float32)
    # possibly save embeddings
    if saveemb:
        try:
            pickle.dump(snapshot_emb, open(savepath, "wb"), -1)
            print("successfully saved embeddings at %s" % savepath)
        except Exception as e:
            print("error saving embeddings:", e)
    return snapshot_emb

In [ ]:
def most_changed_tokens(snapshot_emb, k=10, ignore_zeros=True):
    # find the k tokens whos embedding has changed the most over the whole time period
    snapshots = sorted(snapshot_emb)
    token_sim = []
    for t in snapshot_emb[snapshots[0]].input_model.index2token:
        # ignore the zero embeddings
        if ignore_zeros:
            token_emb = np.vstack([snapshot_emb[s][t] for s in snapshots if np.any(snapshot_emb[s][t] != 0)])
        else:
            token_emb = np.vstack([snapshot_emb[s][t] for s in snapshots])
        # overall sim = min/mean of upper triangular similarity values 
        # -> take into account similarity of all emb to one another at all time points
        if token_emb.shape[0] > 1:
            sim = cosine_similarity(token_emb)
            if ignore_zeros:
                token_sim.append(sim[np.triu_indices(sim.shape[0], k=1)].min())
            else:
                token_sim.append(sim[np.triu_indices(sim.shape[0], k=1)].mean())
        else:
            token_sim.append(1)
    # sort index from smallest to largest - the more different the word, the smaller the sim
    token_idx = np.argsort(token_sim)
    tokens = [(snapshot_emb[snapshots[0]].input_model.index2token[i], token_sim[i]) for i in token_idx]
    return [t for t in tokens if t[0].isalnum()][:k]


def analyze_emb_over_time(snapshot_emb, token, k=5, savefigs=False):
    snapshots = sorted(snapshot_emb)
    # get the two snapshots where the embeddings of the token are the most different
    snapshots_nonz = [s for s in snapshots if np.any(snapshot_emb[s][token] != 0)]
    if len(snapshots_nonz) > 1:
        token_emb = np.vstack([snapshot_emb[s][token] for s in snapshots_nonz])
        sim = cosine_similarity(token_emb)
        rowidx, colidx = np.triu_indices(sim.shape[0], k=1)
        minidx = sim[rowidx, colidx].argmin()
        first, last = snapshots_nonz[rowidx[minidx]], snapshots_nonz[colidx[minidx]]
    else:
        first, last = snapshots[0], snapshots_nonz[0]
    
    # get the corresponding nearest neighbors
    nn_first = snapshot_emb[first].get_nneighbors(token, k, include_simscore=False)
    nn_last = snapshot_emb[last].get_nneighbors(token, k, include_simscore=False)
    
    # get colors for plots later
    colors = {}
    colors[token] = (0., 0., 0., 1.)
    colors["%s (%s)" % (token, first)] = (0., 0., 0., 1.)
    colors["%s (%s)" % (token, last)] = (0., 0., 0., 1.)
    cmap = plt.get_cmap("RdBu")
    for i, t in enumerate(nn_first):
        colors[t] = cmap(0.4*(i/(k-1)))
    for i, t in enumerate(nn_last):
        if t not in colors:
            colors[t] = cmap(1-0.4*(i/(k-1)))
    # plotly colors (careful - wants css colors)
    color_plotly = {t: "rgb(%i,%i,%i)" % tuple(k*255 for k in v[:3]) for t, v in colors.items()}
    # make sure nn_last only contains tokens not in nn_first
    nn_last = [t for t in nn_last if t not in nn_first]
    
    # create embedding matrices per token over time
    token_emb = {}
    for t in [token] + nn_first + nn_last:
        token_emb[t] = np.vstack([snapshot_emb[s][t] for s in snapshots])
        
    # compute similarity of each nn to the token
    sim_scores = {}
    for t in nn_first + nn_last:
        sim_scores[t] = np.diag(cosine_similarity(token_emb[token], token_emb[t]))
    # similarity of token itself to beginning and end embedding
    sim_scores["%s (%s)" % (token, first)] = cosine_similarity(token_emb[token], token_emb[token][[0]]).flatten()
    sim_scores["%s (%s)" % (token, last)] = cosine_similarity(token_emb[token], token_emb[token][[-1]]).flatten()
                
    # plot evolution of similarity scores over time
    snapshot_dates = list(range(len(snapshots)))
    plot_tokens = ["%s (%s)" % (token, first), "%s (%s)" % (token, last)] + nn_first + nn_last
    plt.figure(figsize=(8, 5))
    for t in plot_tokens:
        plt.plot(snapshot_dates, sim_scores[t], "--" if t == "%s (%s)" % (token, first) else "-", color=colors[t], label=t)
    l = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0., fontsize=14)
    plt.xticks(snapshot_dates, snapshots, rotation=90 if len(snapshots) > 5 else 0)
    plt.ylabel("cosine similarity")
    # plt.title(token)
    if savefigs:
        plt.savefig("%s_%s_%s_time.pdf" % (token, snapshots[0], snapshots[-1]), dpi=300, bbox_inches="tight", bbox_extra_artists=[l])
    # interactive timelines with plotly
    df_temp = pd.DataFrame({
        "snapshot date": [datetime.strptime(s, '%Y-%m-%d') for t in plot_tokens for s in snapshots],
        "cosine similarity": np.array([sim_scores[t] for t in plot_tokens]).flatten(),
        "token": [t for t in plot_tokens for s in snapshots],
        "line": ["dash" if t == "%s (%s)" % (token, first) else "solid" for t in plot_tokens for s in snapshots]
    })
    fig_time = px.line(df_temp, x="snapshot date", y="cosine similarity", 
                       color="token", color_discrete_map=color_plotly, hover_name="token", 
                       line_dash="line", line_dash_map='identity')
    fig_time.show()

    # plot 2D PCA vis of embeddings
    full_embedding_mat = []
    labels = []
    color_keys = []
    size = []
    for t in [token] + nn_first + nn_last:
        full_embedding_mat.append(token_emb[t])
        labels.extend(["%s (%s)" % (t, s) for s in snapshots])
        color_keys.extend(len(snapshots)*[t])
        size.extend(list(range(1, len(snapshots) + 1)))
    full_embedding_mat = np.vstack(full_embedding_mat)
    X_kpca = KernelPCA(n_components=2, kernel="cosine").fit_transform(full_embedding_mat)
    # with matplotlib
    plt.figure(figsize=(6, 6))
    plt.scatter(x=X_kpca[:, 0], y=X_kpca[:, 1], s=10*np.array(size), c=[colors[t] for t in color_keys], alpha=0.6)
    plt.xticks([], [])
    plt.yticks([], [])
    plt.xlabel("PC 1")
    plt.ylabel("PC 2")
    # plt.title(token)
    if savefigs:
        plt.savefig("%s_%s_%s_pca.pdf" % (token, snapshots[0], snapshots[-1]), dpi=300, bbox_inches="tight")
    # interactive with plotly
    fig_pca = px.scatter(x=X_kpca[:, 0], y=X_kpca[:, 1], color=color_keys, size=np.sqrt(size), color_discrete_map=color_plotly, hover_name=labels)
    fig_pca.update_traces(hovertemplate='%{hovertext}')  # only show our text, no additional info
    fig_pca.show()
    

In [ ]:
# pre and post-corona times in detail
snapshots = ["2019-%02i-31" % i for i in range(6, 13)] + ["2020-%02i-31" % i for i in range(1, 13)]
snapshot_emb = get_emb_snapshots(snapshots, start_date="2019-04-01", min_freq=50, n_tokens=10000, saveemb=True)
# create plots from paper
analyze_emb_over_time(snapshot_emb, "positive", k=5, savefigs=True)

In [ ]:
# see which words have changed the most at some point in the time period
print("\n".join(["%15s (%.4f)" % x for x in most_changed_tokens(snapshot_emb, k=25, ignore_zeros=True)]))

In [ ]:
# see which words are new
print("\n".join(["%15s (%.4f)" % x for x in most_changed_tokens(snapshot_emb, k=25, ignore_zeros=False)]))

In [ ]:
# create plots for a specific token (e.g. one of the most_changed_tokens)
analyze_emb_over_time(snapshot_emb, "positive", k=5)